# 实验结果可视化
## 直接的kp点

In [15]:
import numpy as np
import torch
from os.path import join
import cv2
import matplotlib.pyplot as plt
import json
import pickle

In [10]:
# 读取generate生成的result.npy
path = 'save/0701_o2h_mid/samples_0701_o2h_mid_000200000_seed10_predefined/results.npy'
res = np.load(path,allow_pickle=True).item()
all_hand_motions = torch.tensor(res['all_hand_motions']) # 
all_obj_motions = torch.tensor(res['all_obj_motions']).reshape(-1,30,3,4) 
seqs = res['seqs']
datapath = '/root/code/seqs/0303_data/'

In [3]:
# all_hand_motions.shape
hand_kp = all_hand_motions[:,:,:63].reshape(-1,30,21,3).numpy()
print(hand_kp.shape)

(81, 30, 21, 3)


In [13]:
# all_obj_motions.shape
# seqs

In [16]:
# 获得对应的物体点
obj_path = '/root/code/seqs/object/'
seq = seqs[0]
seq_path = join(datapath,seq)
meta_path = join(seq_path,'meta.pkl')
with open(meta_path,'rb')as f:
    meta = pickle.load(f)
active_obj = meta['active_obj']
obj_verts = torch.tensor(np.load(join(obj_path,active_obj,'resampled_500_trans.npy'))).float() # 500,3
obj_pose = all_obj_motions[0] # 30,3,4

def get_all_frame_obj_verts(obj_verts, obj_pose):
    # tensor, N,3   T,3,4
    nf = obj_pose.shape[0]
    obj_verts = obj_verts.unsqueeze(0).repeat(nf,1,1)
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    new_obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T
    return new_obj_verts

all_obj_verts = get_all_frame_obj_verts(obj_verts, obj_pose)
print(all_obj_verts.shape)


torch.Size([30, 500, 3])


In [19]:
# 可视化

# 模拟你的手部运动数据 (30帧，21个关键点，每个关键点有3个坐标)
hand_data = hand_kp[0]
obj_data = all_obj_verts.numpy()

# 获取关键点连接信息
connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),       # 拇指
    (0, 5), (5, 6), (6, 7), (7, 8),       # 食指
    (0, 9), (9, 10), (10, 11), (11, 12),  # 中指
    (0, 13), (13, 14), (14, 15), (15, 16), # 无名指
    (0, 17), (17, 18), (18, 19), (19, 20)  # 小指
]

view = 1
# 设置视频参数
fps = 6
frame_size = (3840, 2160)
output_file = 'save/0701_o2h_mid/samples_0701_o2h_mid_000200000_seed10_predefined/0_hand_keypoints.mp4'
calib_path = "/root/code/seqs/calibration_all.json"
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))
R = camera_pose[:3,:3]
T = camera_pose[:3,3]


# 初始化视频写入对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

# 创建一个空白图像
def create_blank_image():
    return np.ones((frame_size[1], frame_size[0], 3), dtype=np.uint8) * 255

# 3D关键点到2D图像坐标的投影
def project_points(points_3d, K, R, T):
    points_3d_hom = np.hstack((points_3d, np.ones((points_3d.shape[0], 1))))  # 转换为齐次坐标
    RT = np.hstack((R, T.reshape(-1, 1)))  # 组合旋转矩阵和平移向量
    points_camera = RT @ points_3d_hom.T  # 投影到相机坐标系
    points_camera = points_camera[:3, :] / points_camera[2, :]  # 归一化
    points_image = K @ points_camera  # 投影到图像平面
    return points_image[:2, :].T

# 绘制关键点和连接线
def draw_keypoints_and_connections(frame, keypoints):
    for connection in connections:
        pt1 = tuple(keypoints[connection[0]].astype(int))
        pt2 = tuple(keypoints[connection[1]].astype(int))
        cv2.line(frame, pt1, pt2, (0, 255, 0), 2)
    for keypoint in keypoints:
        cv2.circle(frame, tuple(keypoint.astype(int)), 5, (0, 0, 255), -1)
    return frame

# 绘制物体点
def draw_object_points(frame, points):
    for point in points:
        cv2.circle(frame, tuple(point.astype(int)), 3, (255, 0, 0), -1)
    return frame

# 创建视频帧
for i in range(hand_data.shape[0]):
    img = create_blank_image()
    hand_keypoints_2d = project_points(hand_data[i], K, R, T)
    object_points_2d = project_points(obj_data[i], K, R, T)
    frame = draw_keypoints_and_connections(img, hand_keypoints_2d)
    frame = draw_object_points(frame, object_points_2d)
    out.write(frame)

# 释放视频写入对象
out.release()

### 将上述分段的代码整合起来：批量可视化生成结果中的hand kp和object points

In [24]:
import numpy as np
import torch
from os.path import join
import cv2
import matplotlib.pyplot as plt
import json
import pickle
from tqdm import *

# 创建一个空白图像
def create_blank_image():
    return np.ones((frame_size[1], frame_size[0], 3), dtype=np.uint8) * 255

# 3D关键点到2D图像坐标的投影
def project_points(points_3d, K, R, T):
    points_3d_hom = np.hstack((points_3d, np.ones((points_3d.shape[0], 1))))  # 转换为齐次坐标
    RT = np.hstack((R, T.reshape(-1, 1)))  # 组合旋转矩阵和平移向量
    points_camera = RT @ points_3d_hom.T  # 投影到相机坐标系
    points_camera = points_camera[:3, :] / points_camera[2, :]  # 归一化
    points_image = K @ points_camera  # 投影到图像平面
    return points_image[:2, :].T

# 绘制关键点和连接线
def draw_keypoints_and_connections(frame, keypoints):
    for connection in connections:
        pt1 = tuple(keypoints[connection[0]].astype(int))
        pt2 = tuple(keypoints[connection[1]].astype(int))
        cv2.line(frame, pt1, pt2, (0, 255, 0), 2)
    for keypoint in keypoints:
        cv2.circle(frame, tuple(keypoint.astype(int)), 5, (0, 0, 255), -1)
    return frame

# 绘制物体点
def draw_object_points(frame, points):
    for point in points:
        cv2.circle(frame, tuple(point.astype(int)), 3, (255, 0, 0), -1)
    return frame

def get_all_frame_obj_verts(obj_verts, obj_pose):
    # tensor, N,3   T,3,4
    nf = obj_pose.shape[0]
    obj_verts = obj_verts.unsqueeze(0).repeat(nf,1,1)
    # return obj_verts
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    new_obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T
    return new_obj_verts


path = 'save/0701_o2h_mid/samples_0701_o2h_mid_000200000_seed10_predefined/results.npy'
obj_path = '/root/code/seqs/object/'
res = np.load(path,allow_pickle=True).item()
all_hand_motions = torch.tensor(res['all_hand_gt']) 
# all_hand_motions = torch.tensor(res['all_hand_motions']) 
all_obj_motions = torch.tensor(res['all_obj_motions']).reshape(-1,30,3,4) 
seqs = res['seqs']
datapath = '/root/code/seqs/0303_data/'

hand_kp = all_hand_motions[:,:,:63].reshape(-1,30,21,3).numpy()
print(hand_kp.shape)

for i in tqdm(range(hand_kp.shape[0])):
    # 获得对应的物体点
    seq = seqs[i] 
    seq_path = join(datapath,seq)
    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    active_obj = meta['active_obj']
    obj_verts = torch.tensor(np.load(join(obj_path,active_obj,'resampled_500_trans.npy'))).float() # 500,3

    obj_pose = all_obj_motions[i] # 30,3,4
    all_obj_verts = get_all_frame_obj_verts(obj_verts, obj_pose)
    print(all_obj_verts.shape)

    # 可视化
    # 模拟你的手部运动数据 (30帧，21个关键点，每个关键点有3个坐标)
    hand_data = hand_kp[i]
    obj_data = all_obj_verts.numpy()

    # 获取关键点连接信息
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 4),       # 拇指
        (0, 5), (5, 6), (6, 7), (7, 8),       # 食指
        (0, 9), (9, 10), (10, 11), (11, 12),  # 中指
        (0, 13), (13, 14), (14, 15), (15, 16), # 无名指
        (0, 17), (17, 18), (18, 19), (19, 20)  # 小指
    ]

    view = 1
    # 设置视频参数
    fps = 6
    frame_size = (3840, 2160)
    output_file = f'save/0701_o2h_mid/samples_0701_o2h_mid_000200000_seed10_predefined/{str(i)}_gt.mp4'
    calib_path = "/root/code/seqs/calibration_all.json"
    with open(calib_path) as f:
        calib_dome = json.load(f)
        f.close()
    camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
    K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))
    R = camera_pose[:3,:3]
    T = camera_pose[:3,3]

    # 初始化视频写入对象
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

    # 创建视频帧
    for i in range(hand_data.shape[0]):
        img = create_blank_image()
        hand_keypoints_2d = project_points(hand_data[i], K, R, T)
        object_points_2d = project_points(obj_data[i], K, R, T)
        frame = draw_keypoints_and_connections(img, hand_keypoints_2d)
        frame = draw_object_points(frame, object_points_2d)
        out.write(frame)

    # 释放视频写入对象
    out.release()


(81, 30, 21, 3)


  0%|          | 0/81 [00:00<?, ?it/s]

torch.Size([30, 500, 3])


  1%|          | 1/81 [00:01<01:45,  1.32s/it]

torch.Size([30, 500, 3])


...

torch.Size([30, 500, 3])


 48%|████▊     | 39/81 [00:50<00:54,  1.29s/it]

torch.Size([30, 500, 3])


 48%|████▊     | 39/81 [00:51<00:55,  1.33s/it]


KeyboardInterrupt: 